# Gibbs Sampling

Στη Στατιστική και τη Μηχανική Μάθηση συχνά συναντούμε πολυδιάστατες πιθανοκατανομές των οποίων η απευθείας δειγματοληψία μπορεί να είναι ιδιαίτερα δύσκολη. Η μέθοδος Gibbs sampling είναι μια μέθοδος Μόντε Κάρλο αλυσίδων Markov (Markov Chain Monte Carlo – MCMC) που επιτρέπει τη δειγματοληψία από πολύπλοκες κατανομές ενημερώνοντας τις μεταβλητές διαδοχικά από τις οριακές τους κατανομές υπό συνθήκη (conditional distributions).

Παρακάτω δίνεται ένας ενδεικτικός κώδικας σε Python που μπορείτε να τρέξετε. Αυτός ο κώδικας εκτελεί Gibbs Sampling για μια 4-διάστατη Κανονική (Gaussian) κατανομή με μη συμμετρικό διάνυσμα μέσων (mean vector).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------
# Function to return conditional mean and variance for x[i]
# given the others in a multivariate normal N(mu, Sigma).
# ------------------------------------------------------------------------
def conditional_params(i, x, mu, Sigma):
    """
    Given a d-dimensional MVN with mean mu and covariance Sigma,
    return the conditional mean and variance for x[i] given x[j != i].

    Parameters:
    -----------
    i : int
        Index of the variable we want to update.
    x : array-like, shape (d,)
        Current values of the entire state vector [x1, x2, ..., xd].
    mu : array-like, shape (d,)
        Mean of the MVN distribution.
    Sigma : array-like, shape (d, d)
        Covariance matrix of the MVN distribution.

    Returns:
    --------
    cond_mean : float
        Conditional mean of x[i] given the other coordinates.
    cond_var : float
        Conditional variance of x[i] given the other coordinates.
    """
    d = len(mu)
    sub_idx = [j for j in range(d) if j != i]  # indices except i

    # Partition the covariance matrix
    Sigma_ii = Sigma[i, i]  # scalar
    Sigma_i_sub = Sigma[i, sub_idx]  # row i, columns != i
    Sigma_sub_i = Sigma[sub_idx, i]  # row != i, column i
    Sigma_sub_sub = Sigma[sub_idx][:, sub_idx]  # submatrix

    inv_sub = np.linalg.inv(Sigma_sub_sub)

    # Conditional mean
    diff = x[sub_idx] - mu[sub_idx]
    cond_mean = mu[i] + Sigma_i_sub @ inv_sub @ diff

    # Conditional variance
    cond_var = Sigma_ii - Sigma_i_sub @ inv_sub @ Sigma_sub_i

    return cond_mean, cond_var

# ------------------------------------------------------------------------
# Main: 4D MVN with non-symmetric mean & covariance
# ------------------------------------------------------------------------
np.random.seed(42)

# Mean vector
mu = np.array([0.0, 1.0, -1.0, 2.0])

# Covariance matrix
Sigma = np.array([
    [1.0,  0.5,  0.0,  0.0],
    [0.5,  1.0,  0.0,  0.0],
    [0.0,  0.0,  1.0,  0.8],
    [0.0,  0.0,  0.8,  2.0]
])

# Number of iterations and burn-in (adjusted to be larger)
iterations = 200000  # e.g., 200K total iterations
burn_in = 80000      # e.g., 80K burn-in

# Initialize x = [x1, x2, x3, x4]
x = np.zeros(4, dtype=float)

# Lists to store samples
samples_x1 = []
samples_x2 = []
samples_x3 = []
samples_x4 = []

for it in range(iterations):
    # We update each coordinate in turn
    for i in range(4):
        cm, cv = conditional_params(i, x, mu, Sigma)
        x[i] = np.random.normal(loc=cm, scale=np.sqrt(cv))

    # Only store samples after burn-in
    if it >= burn_in:
        samples_x1.append(x[0])
        samples_x2.append(x[1])
        samples_x3.append(x[2])
        samples_x4.append(x[3])

samples_x1 = np.array(samples_x1)
samples_x2 = np.array(samples_x2)
samples_x3 = np.array(samples_x3)
samples_x4 = np.array(samples_x4)

# ------------------------------------------------------------------------
# Basic statistics
# ------------------------------------------------------------------------
print("Final x state:", x)
print("\nSample means:")
print("mean(x1) =", np.mean(samples_x1))
print("mean(x2) =", np.mean(samples_x2))
print("mean(x3) =", np.mean(samples_x3))
print("mean(x4) =", np.mean(samples_x4))

print("\nSample variances:")
print("var(x1) =", np.var(samples_x1))
print("var(x2) =", np.var(samples_x2))
print("var(x3) =", np.var(samples_x3))
print("var(x4) =", np.var(samples_x4))

# Empirical correlation matrix
all_samples = np.vstack([samples_x1, samples_x2, samples_x3, samples_x4])
corr_matrix = np.corrcoef(all_samples)
print("\nEmpirical correlation matrix:\n", corr_matrix)

# ------------------------------------------------------------------------
# Histograms
# ------------------------------------------------------------------------
plt.hist(samples_x1, bins=50, density=True)
plt.title("Histogram of x1")
plt.show()

plt.hist(samples_x2, bins=50, density=True)
plt.title("Histogram of x2")
plt.show()

plt.hist(samples_x3, bins=50, density=True)
plt.title("Histogram of x3")
plt.show()

plt.hist(samples_x4, bins=50, density=True)
plt.title("Histogram of x4")
plt.show()

# ------------------------------------------------------------------------
# Trace plots
# ------------------------------------------------------------------------
plt.plot(samples_x1)
plt.title("Trace Plot of x1")
plt.xlabel("Sample Index")
plt.ylabel("x1")
plt.show()

plt.plot(samples_x2)
plt.title("Trace Plot of x2")
plt.xlabel("Sample Index")
plt.ylabel("x2")
plt.show()

plt.plot(samples_x3)
plt.title("Trace Plot of x3")
plt.xlabel("Sample Index")
plt.ylabel("x3")
plt.show()

plt.plot(samples_x4)
plt.title("Trace Plot of x4")
plt.xlabel("Sample Index")
plt.ylabel("x4")
plt.show()

# Ερωτήσεις

1. Περιγράψτε με απλά λόγια πώς λειτουργεί ο αλγόριθμος Gibbs. Ποια είναι τα βασικά του βήματα και σε τι είδους προβλήματα εφαρμόζεται συνήθως;

2. Γιατί στο Gibbs Sampling κοιτάμε την εξέλιξη μόνο μιας μεταβλητής του διανυσματικού χώρου με τις άλλες μεταβλητές να αποτελούν τη συνθήκη (τελευταίες τιμές); Τι πλεονεκτήματα προσφέρει αυτό σε σχέση με την απευθείας (ολική) δειγματοληψία από την κοινή κατανομή;

3. Αναφέρετε δύο παράγοντες που ενδέχεται να καθυστερήσουν ή να εμποδίσουν τη σύγκλιση της αλυσίδας. Ποιες πιθανές βελτιώσεις (ή παραλλαγές του Gibbs Sampling) μπορούν να αντιμετωπίσουν τέτοια προβλήματα;

4. Αναφέρετε τυχόν κριτήρια σύγκλισης της διαδικασίας Gibbs (χωρίς να κάνετε συγκεκριμένους υπολογισμούς).